# Göra en karta över Sveriges kommuner med `spmap`

*Av Anders Sundell*

Alla gillar kartor. De är fina att titta på, och kan förmedla information på ett effektivt och slående sätt - i alla fall ibland. Om det inte finns några geografiska mönster är de inte nödvändigtvis lämpliga. Men om man har regional data kan det vara bra att åtminstone titta på en karta för att se om det finns något mönster där.

I Stata finns det ett bra användarbyggt kommando för att göra kartor, `spmap`. För att göra en karta krävs tre ingredienser:

1. Data med någon geografisk indelning
2. En koordinatfil som visar hur kartan ska ritas
3. Ett id-nummer som matchar ihop de geografiska enheterna i datafilen med koordinatfilen.

Arbetsordningen är som följer:

1. Installera kommandot spmap med `ssc install spmap`. Det här behöver man bara göra en gång.
2. Öppna datafilen med datan som man vill rita ut.
3. Använd kommandot `merge` ([se separat guide](merge_sv.html)) för att lägga till id-numret till de geografiska enheterna.
4. Använd kommandot `spmap` för att rita kartan.

I det här exemplet kommer jag att visa hur man gör en karta över Sveriges kommuner, där färgen visar ett värde, i det här fallet hur många som bor i kommunen. Om du vill göra en egen karta behöver du ladda ned de tre filerna nedan. Spara dem i din projektmapp.

[Fil med id-nummer](../data/spmapid.dta)<br>
[Koordinatfil för vanlig Sverigekarta](../data/swecoord_normal.dta)<br>
[Koordinatfil för Sverigekarta med förstorade storstadsregioner](../data/swecoord_cutouts.dta)

Och som exempeldata kan du även ladda ned den här filen, som innehåller data på befolkningen i Sveriges kommuner 2018.<br>
[Befolkningsdata för Sveriges kommuner 2018](../data/municipality_population2018.dta)

## Förberedelser
Vi börjar med att installera kommandot spmap:

In [1]:
ssc install spmap, replace

checking spmap consistency and verifying not already installed...
all files already exist and are up to date.


Därefter anger vi projektmappen med `cd` (den ser givetvis annorlunda ut på din egen dator) och öppnar sedan exempeldatan över befolkningen i Sveriges kommuner.

In [7]:
cd "/Users/xsunde/Dropbox/Jupyter/stathelp/data"
use "municipality_population2018.dta", clear


/Users/xsunde/Dropbox/Jupyter/stathelp/data



I datafilen har vi tre variabler: municipality_name (kommunens namn), municipality_code (kommunens kod) och pop2018 (befolkningen 2018). Kommunkoden är en [standardiserad kod](https://www.scb.se/hitta-statistik/regional-statistik-och-kartor/regionala-indelningar/lan-och-kommuner/lan-och-kommuner-i-kodnummerordning/) som används av bland annat SCB.

## Sammanfoga datan med id-nummer

Nu ska vi slå ihop datan vi är intresserad av med de speciella id-nummer som hjälper oss att rita kartan. De är inte standardiserade, utan är specifika för varje kartfil. I filen med id-nummer jag har förberett finns det två olika nummer: ett för att rita en "vanlig" karta (spmapid_swemap_normal), och ett för att rita en karta där storstadsregionerna har lyfts ut och förstorats (spmapid_swemap_cutouts). Fördelen med det senare alternativet är att det då blir lättare att se de till ytan små, men till befolkning stora kommunerna kring framförallt Stockholm.

Vi använder oss av kommandot `merge 1:1`, och matchar ihop datafilerna med hjälp av variabeln municipality_code.

In [8]:
merge 1:1 municipality_code using "spmapid.dta"


    Result                           # of obs.
    -----------------------------------------
    not matched                             0
    matched                               290  (_merge==3)
    -----------------------------------------


Vi ser att alla 290 kommuner matchades ihop korrekt. Datan har nu fått två nya variabler: spmapid_swemap_normal och spmapid_swemap_cutouts. (plus en specialvariabel som visar resultatet av matchningen).

## Rita upp en vanlig karta

Nu är vi redo att göra den första kartan! Principen är att vi skriver `spmap` följt av variabeln vi vill rita ut. I det här fallet är variabeln pop2018. Färgerna på kartan kommer då indikera hur många invånare det är i kommunen. Efter variabelnamnet skriver vi `using` och namnet till **koordinatfilen** för kartan. I det här fallet är det "swecoord_normal.dta". Sedan sätter vi ett kommatecken för att börja med options, och anger det viktigaste option: `id(spmapid_swemap_normal)`. Det berättar för Stata att vi vill att varje kommun ska matchas ihop med den kartform som anges i variabeln spmapid_swemap_normal. Vi testar det nedan:

In [12]:
spmap pop2018 using "swecoord_normal.dta", id(spmapid_swemap_normal)

<img src="../images/spmap_1.png" width=200>

Inte så tokigt! Den kan man sedan exportera till exempel som en .png-fil eller .pdf-fil genom att skriva:

In [ ]:
graph export "populationmap.png", replace

## Rita upp en karta med förstorade storstadsregioner

Om vi nu tycker att det är svårt att se vad som händer i storstadsregionerna kan vi använda den andra koordinatfilen. Observera att det här inte är något vi alltid kan göra, utan det kräver alltså att vi har en speciell koordinatfil, som är speciellt anpassad för detta.

Men vi gör då på samma sätt som tidigare, fast nu använder vi det andra id-numret, det som heter spmapid_swemap_cutouts, och den andra koordinatfilen, som heter swecoord_cutouts.dta.

In [14]:
spmap pop2018 using "swecoord_cutouts.dta", id(spmapid_swemap_cutouts)

<img src="../images/spmap_2.png" width=300>

Nu är det lättare att se storstadsregionerna, men det är viktigt att komma ihåg att de nu också har lyfts ut ur den ursprungliga kartan. Dvs det är ett hål där områdena kring Stockholm, Göteborg och Malmö ska vara. Du får själv avgöra om det blir mer eller mindre pedagogiskt, det kan variera från fall till fall.

## Ställa in kartan: Färger, klassindelningar, förklaring

Nu gäller det att göra kartan mer visuellt tilltalande, och enklare att tolka. Det första vi kan göra är att byta färgschema med option `fcolor`. Det finns en hel massa färdiga färgscheman, till exempel "Blues", "Reds", "Greens", "Heat" och "Terrain" för att nämna några. Skriv `help spmap` och klicka sedan på colorlist för att se den fullständiga listan. Vilket färgschema som passar bäst beror naturligtvis på vad det är för information vi vill visa. Här använder vi oss av "Blues", där blåare fält visar högre värden.

In [15]:
spmap pop2018 using "swecoord_cutouts.dta", id(spmapid_swemap_cutouts) fcolor(Blues)

. spmap pop2018 using "swecoord_cutouts.dta", id(spmapid_swemap_cutouts) fcolor
> (Blues)


<img src="../images/spmap_3.png" width=300>

Snyggare! Men vi behöver göra något åt klassindelningen och förklaringen. Längst ner till vänster kan vi se vad färgerna betyder: den ljusaste kategorin visar att det bor mellan 2450 och 10011 personer i kommunen, till exempel. Inte särskilt intuitivt.

Vi kan antingen välja hur många olika klasser vi vill dela in i med hjälp av option `clnumber(5)` eller `clnumber(7)` till exempel, och låta Stata sköta indelningen automatiskt. Då gör Stata så att det hamnar lika många enheter i varje klass. Eller så kan vi själva specifiera vart vi vill dra gränserna. Det gör vi då genom att ange både att vi vill bestämma själva med `clmethod(custom)` och sedan var vi vill dra gränserna, med `clbreaks(0 10000 50000 100000 1000000)`. Här säger vi att vi vill ha tre kategorier: 0-10000, 10000-50000 och 100000-1000000. De tre /// i kommandot är bara för att göra en radbrytning i själva do-filen.

Vi lägger även till option `legend(position(11))` för att ange vart förklaringen ska placeras. Siffran motsvarar siffrorna på en klocka. 12 är rakt upp, 6 är rakt ner, 11 uppe till vänster, osv.

In [21]:
spmap pop2018 using "swecoord_cutouts.dta", id(spmapid_swemap_cutouts) fcolor(Blues) clmethod(custom) ///
clbreaks(0 10000 50000 100000 1000000) legend(position(12))

. spmap pop2018 using "swecoord_cutouts.dta", id(spmapid_swemap_cutouts) fcolor
> (Blues) clmethod(custom) clbreaks(0 10000 50000 100000 1000000) legend(positi
> on(12))


<img src="../images/spmap_4.png" width=300>

## Avslutning

Det här var bara några av sätten att anpassa sin karta för att få den att bli maximalt tydlig och visuellt tilltalande. Kolla igenom `help spmap` för att se vad som mer går att göra. Om man till exempel har några få värden kan man använda sig av `clmethod(unique)` för att visa varje enskilt värde med en egen färg. Välj färgschema utifrån datatyp - här var det ett schema i bara en färg, men det finns sådana som går från en färg till en annan, eller har kontrasterande färger, osv.

Men man behöver som sagt en koordinatfil för att kunna göra kartor med `spmap`. Det går att göra egna sådana utifrån shapefiler (.shp) med hjälp av kommandot `shp2dta`, men det kräver lite mer trix, vilket får bli föremål för en egen guide.